# Homework 5 - The Marvel Universe!

In [ ]:
# importing libraries that we needed.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tkinter import _flatten

## 1. Data

We imported the datasets for the questions in ADM HW-5.

In [3]:
nodes=pd.read_csv('nodes.csv')

In [4]:
edges=pd.read_csv('edges.csv')

In [5]:
hero_net=pd.read_csv('hero-network.csv')

In [7]:
#Contains two columns (node, type), indicating the nodes' name and type (comic, hero).

nodes.head(5)

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


In [8]:
#Contains two columns (hero, comic), indicating which comics the heroes appear in.

edges.head(5)

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [9]:
#Contains the network of heroes who have appeared together in the comics.

hero_net.head(5)

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [38]:
#SLACK 
#in the edges.csv file the name of Spider-Man was mistyped, 
#so you will not get the “hero” type attribute for the node if you don’t correct the typo manually

edges.loc[edges['hero'].str.contains("Spider-M", case=False)]

,hero,comic
75528,SPIDER-MAN CLONE/BEN,A 400
75529,SPIDER-MAN CLONE/BEN,ASM 149
75530,SPIDER-MAN CLONE/BEN,ASM 151
75531,SPIDER-MAN CLONE/BEN,ASM 393
75532,SPIDER-MAN CLONE/BEN,ASM 394
...,...,...
77294,SPIDER-MAN/PETER PARKER,XFOR 4
77295,SPIDER-MAN/PETER PARKER,XM 35
77296,SPIDER-MAN/PETER PARKER,XM 37
77297,SPIDER-MAN/PETER PARKER,XM 38


## Data preprocessing

#### 1-To overcome inconsistencies in the data, 

#### a-We deleted extra spaces at the end of some heroes' names in "hero_network.csv" file, compared to their names in "edges.csv".

In [10]:
hero_net.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [11]:
print("in the column 'hero1', Number of rows that has an extra space at the end of the hero name is",len(hero_net.loc[hero_net['hero1'].str.endswith(' ')]))

in the column 'hero1', Number of rows that has an extra space at the end of the hero name is 75735


In [12]:
print("in the column 'hero2', Number of rows that has an extra space at the end of the hero name is",len(hero_net.loc[hero_net['hero2'].str.endswith(' ')]))

in the column 'hero2', Number of rows that has an extra space at the end of the hero name is 73556


We created a new dataframe after preprocessing, named as "hero_net_pp".

In [13]:
hero_net_pp = pd.DataFrame(columns=['hero1', 'hero2'])

hero_net_pp['hero1']=hero_net['hero1'].str.rstrip(' ')
hero_net_pp['hero2']=hero_net['hero2'].str.rstrip(' ')

In [14]:
print("in the column 'hero1', Number of rows that has an extra space at the end of the hero name is",len(hero_net_pp.loc[hero_net_pp['hero1'].str.endswith(' ')]))

in the column 'hero1', Number of rows that has an extra space at the end of the hero name is 0


In [15]:
print("in the column 'hero2', Number of rows that has an extra space at the end of the hero name is",len(hero_net_pp.loc[hero_net_pp['hero2'].str.endswith(' ')]))

in the column 'hero2', Number of rows that has an extra space at the end of the hero name is 0


#### b- We deleted extra "/" at the end of some heroes' names in "hero_network.csv" file, compared to their names in "edges.csv"

In [16]:
print("in the column 'hero1', Number of rows that has an extra space at the end of the hero name is",len(hero_net_pp.loc[hero_net_pp['hero1'].str.endswith('/')]))

in the column 'hero1', Number of rows that has an extra space at the end of the hero name is 15323


In [17]:
print("in the column 'hero2', Number of rows that has an extra space at the end of the hero name is",len(hero_net_pp.loc[hero_net_pp['hero2'].str.endswith('/')]))

in the column 'hero2', Number of rows that has an extra space at the end of the hero name is 15520


In [18]:
hero_net_pp['hero1']=hero_net_pp['hero1'].str.rstrip('/')
hero_net_pp['hero2']=hero_net_pp['hero2'].str.rstrip('/')

In [19]:
print("in the column 'hero1', Number of rows that has an extra space at the end of the hero name is",len(hero_net_pp.loc[hero_net_pp['hero1'].str.endswith('/')]))
print("in the column 'hero2', Number of rows that has an extra space at the end of the hero name is",len(hero_net_pp.loc[hero_net_pp['hero2'].str.endswith('/')]))

in the column 'hero1', Number of rows that has an extra space at the end of the hero name is 0
in the column 'hero2', Number of rows that has an extra space at the end of the hero name is 0


#### c- The hero name 'SPIDER-MAN/PETER PAR' in 'hero-network.csv' (due to a string length limit), has been changed  as 'SPIDER-MAN/PETER PARKER' as like as in 'edges.csv' 

In [20]:
hero_net_pp.loc[hero_net_pp['hero1'] == 'SPIDER-MAN/PETER PAR'] 

,hero1,hero2
398,SPIDER-MAN/PETER PAR,FIRESTAR/ANGELICA JO
399,SPIDER-MAN/PETER PAR,THUNDERBALL/DR. ELIO
400,SPIDER-MAN/PETER PAR,NAMORITA/NITA PRENTI
401,SPIDER-MAN/PETER PAR,SPEEDBALL/ROBBIE BAL
402,SPIDER-MAN/PETER PAR,HULK/DR. ROBERT BRUC
...,...,...
574356,SPIDER-MAN/PETER PAR,MAGMA/AMARA AQUILLA
574357,SPIDER-MAN/PETER PAR,SCARLET WITCH/WANDA
574358,SPIDER-MAN/PETER PAR,MAGIK/ILLYANA RASPUT
574359,SPIDER-MAN/PETER PAR,CAPTAIN AMERICA


In [21]:
hero_net_pp = hero_net_pp.apply(lambda x: x.replace({'SPIDER-MAN/PETER PAR':'SPIDER-MAN/PETER PARKER'}, regex=True))

In [22]:
#to check the dataset, after we changed heroes names. 
hero_net_pp.loc[hero_net_pp['hero2'] == 'SPIDER-MAN/PETER PARKER'] 

,hero1,hero2
397,NOVA/RICHARD RIDER,SPIDER-MAN/PETER PARKER
1505,"STACY, PAUL",SPIDER-MAN/PETER PARKER
1510,"STACY, JILL",SPIDER-MAN/PETER PARKER
1514,"STACY, ARTHUR",SPIDER-MAN/PETER PARKER
1517,"PARKER, MAY",SPIDER-MAN/PETER PARKER
...,...,...
574271,NIGHTCRAWLER/KURT WA,SPIDER-MAN/PETER PARKER
574291,SUNSPOT/ROBERTO DACO,SPIDER-MAN/PETER PARKER
574310,STORM/ORORO MUNROE S,SPIDER-MAN/PETER PARKER
574328,MIRAGE II/DANIELLE M,SPIDER-MAN/PETER PARKER


#### 2- Some entries in the 'hero-network.csv' have the same hero in both columns. In the graph, because a self-loop makes no sense in this network, We removed those from the dataset.

In [23]:
hero_net_pp.loc[hero_net_pp['hero1'] == hero_net_pp['hero2']].value_counts() 

hero1                 hero2               
PATRIOT/JEFF MACE     PATRIOT/JEFF MACE       1275
MISS AMERICA/MADELIN  MISS AMERICA/MADELIN     672
BUCKY/BUCKY BARNES    BUCKY/BUCKY BARNES       120
CAPTAIN AMERICA       CAPTAIN AMERICA          120
WHIZZER/ROBERT L. FR  WHIZZER/ROBERT L. FR      37
RAVONNA LEXUS RENSLA  RAVONNA LEXUS RENSLA       2
BLUE DIAMOND/PROFESS  BLUE DIAMOND/PROFESS       1
DR. STRANGE DOPPELGA  DR. STRANGE DOPPELGA       1
FIN/PETER NOBLE       FIN/PETER NOBLE            1
SILVER SCORPION/ELIS  SILVER SCORPION/ELIS       1
SUB-MARINER/NAMOR MA  SUB-MARINER/NAMOR MA       1
VINDICATOR II DOPPEL  VINDICATOR II DOPPEL       1
dtype: int64

In [24]:
#Removing self loop
hero_net_pp.drop(hero_net_pp.index[(hero_net_pp["hero1"] == hero_net_pp['hero2'] )],axis=0,inplace=True)

In [25]:
hero_net_pp.loc[hero_net_pp['hero1'] == hero_net_pp['hero2']].value_counts() 

Series([], dtype: int64)

### Graphs setup

#### First graph
Will be constructed using the data stored in the 'hero-network.csv' file, in which an edge between two heroes can be found if they have appeared in the same comic together. The number of edges between two heroes represents the number of times they have collaborated in different comics. 

##### The graph should be considered weighted and undirected. 
It is up to you to decide which metric to use to calculate the weights, but we anticipate that the cost will be lower for heroes with more collaborations. Please specify which metric you used to select the weights in the report.


In [44]:
#import networkx as nx
G = nx.Graph()

In [28]:
hero_net_pp.loc[hero_net_pp['hero1'] == 'SPIDER-MAN/PETER PAR'] 

,hero1,hero2
398,SPIDER-MAN/PETER PAR,FIRESTAR/ANGELICA JO
399,SPIDER-MAN/PETER PAR,THUNDERBALL/DR. ELIO
400,SPIDER-MAN/PETER PAR,NAMORITA/NITA PRENTI
401,SPIDER-MAN/PETER PAR,SPEEDBALL/ROBBIE BAL
402,SPIDER-MAN/PETER PAR,HULK/DR. ROBERT BRUC
...,...,...
574356,SPIDER-MAN/PETER PAR,MAGMA/AMARA AQUILLA/
574357,SPIDER-MAN/PETER PAR,SCARLET WITCH/WANDA
574358,SPIDER-MAN/PETER PAR,MAGIK/ILLYANA RASPUT
574359,SPIDER-MAN/PETER PAR,CAPTAIN AMERICA


#### Second graph 
The data in 'nodes.csv' and 'edges.csv' will be used to construct the second graph. The type of node (hero/comic) can be found in 'nodes.csv', and an edge between a hero node and a comic node can be found in 'edges.csv' when the hero has appeared in that specific comic.

##### This graph is assumed to be undirected and unweighted.

In [52]:
#Hero=3-D MAN/CHARLES CHAN

edges.loc[edges['hero'] == '3-D MAN/CHARLES CHAN']

,hero,comic
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
5,3-D MAN/CHARLES CHAN,H2 252
6,3-D MAN/CHARLES CHAN,M/PRM 35
7,3-D MAN/CHARLES CHAN,M/PRM 36
8,3-D MAN/CHARLES CHAN,M/PRM 37
9,3-D MAN/CHARLES CHAN,WI? 9


## 2. Backend Implementation

### Functionality 1 - extract the graph's features

### Functionality 2 - Find top superheroes

### Functionality 3 - Shortest ordered Route

### Functionality 4 - Disconnecting Graphs

### Functionality 5 - Extracting Communities

## 3. Frontend Implementation

In [ ]:
#In the visualization part, you can use any available function (networkx functions are highly recommended).

### Visualization 1 - Visualize some features of the network

### Visualization 2 - Visualize centrality measure